In [1]:
cd GUI

/home/pyo/GUI


In [ ]:
import os
import sys
import joblib
from PIL import ImageOps
from face_recognition import preprocessing
from inference.util import draw_bb_on_img
from sys import exit
import cv2
import PyQt5
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage
from PyQt5.QtGui import QPixmap
from PyQt5 import QtGui, QtCore
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import QApplication
from PyQt5.QtWidgets import QDialog
from PyQt5.uic import loadUi
from functools import partial
from PIL.ImageQt import ImageQt
from PIL import Image
import numpy as np
import time
from mmdet.apis import inference_detector, init_detector
from mmcv.utils import is_str
import random
import mmcv
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import skimage
from skimage import io
import torchvision
import eval_widerface
import torchvision_model
import model


model = torch.load('save_model/DynamicRCNN_model.pt')
score_thr = 0.7
CLASSES = ('ID_gum', 'buttering', 'couque_coffee', 'chocopie', 'cidar', 
           'couque_white', 'coke', 'diget_ori', 'diget_choco', 'gumi_gumi', 
           'homerunball', 'jjolbyung_noodle', 'juicyfresh', 'jjolbyung_ori', 
           'spearmint', 'squid_peanut', 'samdasu', 'tuna', 'toreta', 
           'vita500', 'welchs', 'zec')
COST = (990, 1290, 990, 990, 800,
        990, 1100, 1990, 2390, 1090,
        1390, 990, 990,990,
        990, 2690, 690, 2990, 1490,
        690, 990, 1490)
class_by_color = [[128, 0, 0], [255,0,0], [255,165,0], [128,128,0], [124,252,0], [0,128,0], 
                  [47,79,79], [0,255,255], [0,206,209], [100,149,237], [25,25,112], [0,0,255], 
                  [147,112,219], [216,191,216], [199,21,133], [255,105,180], [250,235,215], [160,82,45], 
                  [205,133,63], [255,222,173], [112,128,144], [0,0,0]]
label_list = []
label_length = 0
stop_flag = False
label_dict = {}

label_dict_prev = {}
model_path = 'out/model.pt'
depth = 50
scale = 1.0
gl_name = ""
gl_cost = 0
capture_1 = 0
finish_button_flag = 0

print(os.getcwd())

def random_color(label_num):
    return tuple(class_by_color[label_num])

def imshow_det_bboxes(img,
                      bboxes,
                      labels,
                      class_names=None,
                      score_thr=0,
                      bbox_color='red',
                      text_color='red',
                      thickness=1.5,
                      font_scale=1.5,
                      show=True,
                      win_name='',
                      wait_time=0,
                      out_file=None):

    assert bboxes.ndim == 2
    assert labels.ndim == 1
    assert bboxes.shape[0] == labels.shape[0]
    assert bboxes.shape[1] == 4 or bboxes.shape[1] == 5
    img = mmcv.imread(img)

    if score_thr > 0:
        assert bboxes.shape[1] == 5
        scores = bboxes[:, -1]
        inds = scores > score_thr
        bboxes = bboxes[inds, :]
        labels = labels[inds]

    img = np.ascontiguousarray(img)
    origin_labels = labels
    i = 0
    for bbox, label in zip(bboxes, labels):
        rand_color = random_color(label)
        bbox_int = bbox.astype(np.int32)
        left_top = (bbox_int[0], bbox_int[1])
        right_bottom = (bbox_int[2], bbox_int[3])
        cv2.rectangle(img, left_top, right_bottom, rand_color, thickness=thickness)
        label_text = class_names[label] if class_names is not None else f'cls {label}'
        if len(bbox) > 4:
            label_text += f'|{bbox[-1]:.02f}'
        cv2.putText(img, label_text, (bbox_int[0], bbox_int[1] - 2), cv2.FONT_HERSHEY_COMPLEX, font_scale, rand_color, thickness=2)
        i += 1

    if show:
        imshow(img, win_name, wait_time)
    if out_file is not None:
        imwrite(img, out_file)
    return img, origin_labels

def show_result(img,
                result,
                score_thr=0.3,
                bbox_color='magenta',
                text_color='magenta',
                thickness=3,
                font_scale=1.0,
                win_name='',
                show=False,
                wait_time=0,
                out_file=None):

    img = mmcv.imread(img)
    img = img.copy()
    if isinstance(result, tuple):
        bbox_result, segm_result = result
        if isinstance(segm_result, tuple):
            segm_result = segm_result[0]  # ms rcnn
    else:
        bbox_result, segm_result = result, None
    bboxes = np.vstack(bbox_result)
    labels = [
        np.full(bbox.shape[0], i, dtype=np.int32)
        for i, bbox in enumerate(bbox_result)
    ]
    labels = np.concatenate(labels)

    if segm_result is not None and len(labels) > 0: # non empty
        segms = mmcv.concat_list(segm_result)
        inds = np.where(bboxes[:, -1] > score_thr)[0]
        np.random.seed(42)
        color_masks = [
            np.random.randint(0, 256, (1, 3), dtype=np.uint8)
            for _ in range(max(labels) + 1)
        ]
        for i in inds:
            i = int(i)
            color_mask = color_masks[labels[i]]
            mask = segms[i]
            img[mask] = img[mask] * 0.5 + color_mask * 0.5

    if out_file is not None:
        show = False

    img, detect_labels = imshow_det_bboxes(img,
                                           bboxes,
                                           labels,
                                           class_names=CLASSES,
                                           score_thr=score_thr,
                                           bbox_color=bbox_color,
                                           text_color=text_color,
                                           thickness=thickness,
                                           font_scale=font_scale,
                                           win_name=win_name,
                                           show=show,
                                           wait_time=wait_time,
                                           out_file=out_file)

    if not (show or out_file):
        return img, detect_labels

#sseven
class Yesbutton_class(QDialog):    
    def __init__(self,parent):
        super(Yesbutton_class, self).__init__(parent)
        Yesbutton_ui = 'fiveth(payment).ui'
        loadUi(Yesbutton_ui, self)

        
        self.show()
        
        self.pay_timer = QTimer(self)
        self.pay_timer.timeout.connect(self.pay_window_close)
        self.pay_timer.start(3000)
        
    def pay_window_close(self):
        self.close()
        
        
        
# 결제 화면(5)
class PaymentWindow(QDialog):    
    def __init__(self, parent):
        super(PaymentWindow, self).__init__(parent)
        
        global label_dict
        global gl_cost
        global capture_1
        
      #  payment_ui = '/home/jmh/mmdetection/GUI/fiveth(payment).ui'
        payment_ui = 'seven.ui'
        loadUi(payment_ui, self)
        self.productlist.setColumnCount(3)
        column_headers = ['품목', '수량', '가격']
        self.productlist.setHorizontalHeaderLabels(column_headers)
         
                   
        
        
#         label = self.product_cap
#         cap = ImageQt(capture_1)
#         cap1 = QtGui.QPixmap.fromImage(cap)
#         self.product_cap.setPixmap(cap1)
#         self.product_cap.setScaledContents(True)

        qformat=QImage.Format_Indexed8
        if len(capture_1.shape)==3:
            if capture_1.shape[2]==4:
                qformat=QImage.Format_RGBA888
            else:
                qformat=QImage.Format_RGB888

        outImage=QImage(capture_1,capture_1.shape[1],capture_1.shape[0],capture_1.strides[0],qformat)
        outImage=outImage.rgbSwapped()
        

        
        self.product_cap.setPixmap(QPixmap.fromImage(outImage))
        self.product_cap.setScaledContents(True)
        
        
            
            
            
        self.receipt()
        self.productlist_func()
        self.Yesbutton.clicked.connect(self.Yesbutton_1)
        self.Nobutton.clicked.connect(self.Nobutton_1)
        self.show()
        
    def Yesbutton_1(self):
        Yesbutton_class(self)

    def Nobutton_1(self):
        global stop_flag
        stop_flag = False
        self.close()

        
    def productlist_func(self):
        result = 0
        self.productlist.clearContents()
        
        rowcount = len(list(label_dict.keys()))
        self.productlist.setRowCount(rowcount)
        for ld in range(len(label_dict.keys())):
                col1 = QTableWidgetItem(str(list(label_dict.keys())[ld]))
                self.productlist.setItem(ld,0,col1)

                col2 = QTableWidgetItem(str(list(label_dict.values())[ld]))
                self.productlist.setItem(ld,1,col2)

                cost = list(label_dict.values())[ld] * COST[ld]
                result += cost
                col3 = QTableWidgetItem(str(cost))
                self.productlist.setItem(ld,2,col3)

                self.Totalcost.clear()
                self.Totalcost.addItem("     " + str(gl_cost)+"  Won")
            
    def receipt(self):
        global gl_name
        self.nameLabel.addItem(gl_name)


# 객체 인식 화면(4)
class ObjectDetection(QDialog):
    def __init__(self, parent):
        super(ObjectDetection, self).__init__(parent)
       # detection_ui = '/home/jmh/mmdetection/GUI/fourth(detection).ui'
        detection_ui = 'fourth(detection).ui'
        loadUi(detection_ui, self)
        self.image = None
        self.processedImage = None
        
        self.start_webcam()
        #self.detectButton.setCheckable(True)
        
        self.stopButton.clicked.connect(self.stop_webcam)
        self.face_Enabled = True
        self.tableWidget.setColumnCount(3)
        column_headers = ['품목', '수량', '가격']
        self.tableWidget.setHorizontalHeaderLabels(column_headers)
        self.show()

    def goto_payment(self):
        PaymentWindow(self)

    def start_webcam(self):
        self.capture = cv2.VideoCapture(2)
        self.capture2 = cv2.VideoCapture(4)
       
        self.capture.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
        self.capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        
        self.capture2.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
        self.capture2.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(1)

    def update_frame(self):
        global gl_name
        gl_name = ""
        self.flag = 0
        ret, self.image = self.capture.read()
        ret, self.frame = self.capture2.read()
        
        self.image = cv2.flip(self.image, 1)
        self.frame = cv2.flip(self.frame, 1)
        
        self.frame = Image.fromarray(self.frame)        
        self.displayImage(self.image, 2)
        
        detected_image, detected_labels = self.detect_product(self.image)
        self.detectedImage = detected_image

        self.displayImage(detected_image, 2)
        self.displayLabels(detected_labels)
            
        face_recogniser = joblib.load('model/face_recogniser.pkl')
        preprocess = preprocessing.ExifOrientationNormalize()
        faces = face_recogniser(preprocess(self.frame))
        if  faces != []:
            draw_bb_on_img(faces, self.frame)
            gl_name = faces[0][0][0]
            
            self.displayFace(self.frame)
        
    def displayFace(self, img):
        pix = np.array(img)
        
        qformat=QImage.Format_Indexed8
        if len(pix.shape)==3:
            if pix.shape[2]==4:
                qformat=QImage.Format_RGBA888
            else:
                qformat=QImage.Format_RGB888

        outImage=QImage(pix,pix.shape[1],pix.shape[0],pix.strides[0],qformat)
        outImage=outImage.rgbSwapped()

        
        
        self.imgLabel.setPixmap(QPixmap.fromImage(outImage))
        self.imgLabel.setScaledContents(True)
        
    
    def detect_product(self, img):
        result = inference_detector(model, img)
        img, labels = show_result(img, result, score_thr, show=False)
        
        return img, labels

    def stop_webcam(self):
        # 결제하기 버튼
        global stop_flag
        global gl_name
        global capture_1
        capture_1 = self.detectedImage
        
        if gl_name:
            stop_flag = True
            PaymentWindow(self)
        else :
            pass
 
        
    def displayImage(self, img, window=1):
        qformat=QImage.Format_Indexed8
        if len(img.shape)==3:
            if img.shape[2]==4:
                qformat=QImage.Format_RGBA888
            else:
                qformat=QImage.Format_RGB888

        outImage=QImage(img,img.shape[1],img.shape[0],img.strides[0],qformat)
        outImage=outImage.rgbSwapped()

        if window==1:
            self.imgLabel.setPixmap(QPixmap.fromImage(outImage))
            self.imgLabel.setScaledContents(True)

        if window==2:
            self.processedImgLabel.setPixmap(QPixmap.fromImage(outImage))
            self.processedImgLabel.setScaledContents(True)
            
            
    def displayLabels(self, labels):
        result = 0
        global label_length
        global label_dict_prev
        global stop_flag
        global gl_cost
        global label_dict
        
        if stop_flag == False:
            label_dict = {}
            for l in labels:
                if CLASSES[l] in (label_dict.keys()):

                    label_dict[CLASSES[l]] += 1
                else:
                    label_dict[CLASSES[l]] = 1
            if label_dict.keys() == label_dict_prev.keys() and label_dict.values() == label_dict_prev.values():
                pass
            else:
                self.tableWidget.clearContents()
                rowcount = len(list(label_dict.keys()))
                self.tableWidget.setRowCount(rowcount)
                for ld in range(len(label_dict.keys())):
                        col1 = QTableWidgetItem(str(list(label_dict.keys())[ld]))
                        self.tableWidget.setItem(ld,0,col1)

                        col2 = QTableWidgetItem(str(list(label_dict.values())[ld]))
                        self.tableWidget.setItem(ld,1,col2)

                        cost = list(label_dict.values())[ld] * COST[ld]
                        result += cost
                        col3 = QTableWidgetItem(str(cost))
                        self.tableWidget.setItem(ld,2,col3)

                        self.cost_list.clear()
                        self.cost_list.addItem("     " + str(result)+"  Won")
                        gl_cost = result
            label_dict_prev = label_dict
        else :
            pass
                

        


# 얼굴 및 정보 등록 화면(3)
# class WaitWindow(QWidget):
#     def __init__(self, parent=None):
#         super(WaitWindow).__init__()
#         loadUi('sixth(wait).ui')
#         self.show()

class RegisterInfo(QDialog):
    def __init__(self, parent=None):
        super(RegisterInfo, self).__init__(parent)
        #loadUi('/home/jmh/mmdetection/GUI/third(info).ui', self)
        loadUi('third(info).ui', self)
        
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        ##############################################################
        self.capture = cv2.VideoCapture(4)
        self.capture.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
        self.capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        

        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(1)
        
        self.name = ''
        self.dialogs = list()
        self.finish_button.clicked.connect(self.next_frame)
        self.setWindowTitle('Register Panel')
        self.show()

    def update_frame(self):
        _, self.image = self.capture.read()
        self.image = cv2.flip(self.image, 1)
        self.displayImage(self.image)
        
        self.nameButton.clicked.connect(self.get_name)
        
        self.front_capture_button.clicked.connect(self.capture_front)
        self.left_capture_button.clicked.connect(self.capture_left)
        self.right_capture_button.clicked.connect(self.capture_right)
        self.up_capture_button.clicked.connect(self.capture_up)
        self.down_capture_button.clicked.connect(self.capture_down)
        
        
    def get_name(self):
        self.name = self.nameLabel.toPlainText()
    #    if not(os.path.isdir(os.path.join('/home/jmh/mmdetection/GUI/DB/', self.name))):
     #       os.makedirs(os.path.join('/home/jmh/mmdetection/GUI/DB/', self.name))
        if not(os.path.isdir(os.path.join('DB/', self.name))):
            os.makedirs(os.path.join('DB/', self.name))
    
    def capture_front(self):
        global finish_button_flag
        finish_button_flag += 1
        
        frame = self.image
        qformat = QImage.Format_Indexed8
        if len(frame.shape) == 3:
            if frame.shape[2] == 4:
                qformat = QImage.Format_RGBA888
            else:
                qformat = QImage.Format_RGB888

        outImage = QImage(frame, frame.shape[1], frame.shape[0], frame.strides[0], qformat)
        outImage = outImage.rgbSwapped()
#        outImage.save(os.path.join(os.path.join('/home/jmh/mmdetection/GUI/DB/', self.name), self.name + '_front_img.jpg'))
        outImage.save(os.path.join(os.path.join('DB/', self.name), self.name + '_front_img.jpg'))
    
    
        self.front_label.setPixmap(QPixmap.fromImage(outImage))
        self.front_label.setScaledContents(True)
        
    def capture_left(self):
        global finish_button_flag
        finish_button_flag += 1
        
        frame = self.image
        qformat = QImage.Format_Indexed8
        if len(frame.shape) == 3:
            if frame.shape[2] == 4:
                qformat = QImage.Format_RGBA888
            else:
                qformat = QImage.Format_RGB888

        outImage = QImage(frame, frame.shape[1], frame.shape[0], frame.strides[0], qformat)
        outImage = outImage.rgbSwapped()
#        outImage.save(os.path.join(os.path.join('/home/jmh/mmdetection/GUI/DB/', self.name), self.name + '_left_img.jpg'))
        outImage.save(os.path.join(os.path.join('DB/', self.name), self.name + '_left_img.jpg'))
        

        self.left_label.setPixmap(QPixmap.fromImage(outImage))
        self.left_label.setScaledContents(True)
        
    def capture_right(self):
        global finish_button_flag
        finish_button_flag += 1
        
        frame = self.image
        qformat = QImage.Format_Indexed8
        if len(frame.shape) == 3:
            if frame.shape[2] == 4:
                qformat = QImage.Format_RGBA888
            else:
                qformat = QImage.Format_RGB888

        outImage = QImage(frame, frame.shape[1], frame.shape[0], frame.strides[0], qformat)
        outImage = outImage.rgbSwapped()
        outImage.save(os.path.join(os.path.join('DB/', self.name), self.name + '_right_img.jpg'))
        
        self.right_label.setPixmap(QPixmap.fromImage(outImage))
        self.right_label.setScaledContents(True)
        
    def capture_up(self):
        global finish_button_flag
        finish_button_flag += 1
        
        frame = self.image
        qformat = QImage.Format_Indexed8
        if len(frame.shape) == 3:
            if frame.shape[2] == 4:
                qformat = QImage.Format_RGBA888
            else:
                qformat = QImage.Format_RGB888

        outImage = QImage(frame, frame.shape[1], frame.shape[0], frame.strides[0], qformat)
        outImage = outImage.rgbSwapped()
        outImage.save(os.path.join(os.path.join('DB/', self.name), self.name + '_up_img.jpg'))
        
        self.up_label.setPixmap(QPixmap.fromImage(outImage))
        self.up_label.setScaledContents(True)
        
        
    def capture_down(self):
        global finish_button_flag
        finish_button_flag += 1
        
        frame = self.image
        qformat = QImage.Format_Indexed8
        if len(frame.shape) == 3:
            if frame.shape[2] == 4:
                qformat = QImage.Format_RGBA888
            else:
                qformat = QImage.Format_RGB888

        outImage = QImage(frame, frame.shape[1], frame.shape[0], frame.strides[0], qformat)
        outImage = outImage.rgbSwapped()
#        outImage.save(os.path.join(os.path.join('/home/jmh/mmdetection/GUI/DB/', self.name), self.name + '_down_img.jpg'))
        outImage.save(os.path.join(os.path.join('DB/', self.name), self.name + '_down_img.jpg'))
        
        self.down_label.setPixmap(QPixmap.fromImage(outImage))
        self.down_label.setScaledContents(True)
    
    def next_frame(self):
        global finish_button_flag
        if finish_button_flag >= 5:
            os.system("mmdetection/face-recognition/tasks/train.sh DB/")
            os.system("python mmdetection/face-recognition/generate_embeddings.py --input-folder DB/ --output-folder out/")
            os.system("python mmdetection/face-recognition/train.py -d DB/ -e out/embeddings.txt -l out/labels.txt -c out/class_to_idx.pkl")

            self.show()
            self.timer.stop()
            self.close()
            finish_button_flag = 0
        
        
    def displayImage(self, img):
        qformat = QImage.Format_Indexed8
        if len(img.shape) == 3:
            if img.shape[2] == 4:
                qformat = QImage.Format_RGBA888
            else:
                qformat = QImage.Format_RGB888

        outImage = QImage(img,img.shape[1],img.shape[0],img.strides[0],qformat)
        outImage = outImage.rgbSwapped()

        self.video_frame.setPixmap(QPixmap.fromImage(outImage))
        self.video_frame.setScaledContents(True)


# 얼굴 등록 or 결제 화면(2)
class RegisterWindow(QDialog):
    def __init__(self, parent):
        super(RegisterWindow, self).__init__(parent)
#        register_ui = '/home/jmh/mmdetection/GUI/second(register).ui'
        register_ui = 'second(register).ui'
        loadUi(register_ui, self)

        self.register_button.setFont(QFont("12롯데마트드림Bold.ttf", 15))
        self.payment_button.setFont(QFont("12롯데마트드림Bold.ttf", 15))
        
        self.register_button.clicked.connect(self.faceRegisterButtonFunction)
        self.payment_button.clicked.connect(self.paymentButtonFunction)
        self.show()

    def paymentButtonFunction(self):
        ObjectDetection(self)

    def faceRegisterButtonFunction(self):
        RegisterInfo(self)

# 시작 화면(1)
class MainWindow(QMainWindow) :
    def __init__(self) :
        super().__init__()
        main_ui = 'first(main).ui'
        loadUi(main_ui, self)
        self.showMaximized()

#        self.title_label.setFont(QFont("/home/jmh/mmdetection/GUI/12롯데마트드림Bold.ttf", 30))
#        self.start_button.setFont(QFont("/home/jmh/mmdetection/GUI/12롯데마트드림Bold.ttf", 15))

        self.title_label.setFont(QFont("12롯데마트드림Bold.ttf", 30))
        self.start_button.setFont(QFont("12롯데마트드림Bold.ttf", 15))

        # START 버튼 클릭 시 이벤트 발생
        self.start_button.clicked.connect(self.startButtonFunction)

    def startButtonFunction(self):
        RegisterWindow(self)


if __name__ == "__main__" :
    app = QApplication(sys.argv)
    mainWindow = MainWindow()
    mainWindow.show()
    app.exec_()

/home/pyo/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'mmdet.models.dense_heads.rpn_head.RPNHead' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/pyo/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'mmdet.models.roi_heads.dynamic_roi_head.DynamicRoIHead' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/pyo/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'mmcv.ops.roi_align.RoIAlign' has c

/home/pyo/GUI
